In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import plotly.graph_objects as go

### Import datasets

In [2]:
df = pd.read_csv("../ASA/2005.csv")
airport_df = pd.read_csv("airports.csv")
airport_df.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [3]:
useful_cols = ['Month', 'DayofMonth', 'DepTime', 'CRSDepTime', 'DepDelay', 'ArrTime', 'CRSArrTime', 'ArrDelay', 'TailNum', 
               #'AirTime', "ActualElapsedTime", "CRSElapsedTime", 'Distance', 
               'Origin', 'Dest', 'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'LateAircraftDelay']

In [4]:
df = df[useful_cols]
df.head()

,Month,DayofMonth,DepTime,CRSDepTime,DepDelay,ArrTime,CRSArrTime,ArrDelay,TailNum,Origin,Dest,Cancelled,Diverted,CarrierDelay,WeatherDelay,LateAircraftDelay
0,1,28,1603.0,1605,-2.0,1741.0,1759,-18.0,N935UA,BOS,ORD,0,0,0,0,0
1,1,29,1559.0,1605,-6.0,1736.0,1759,-23.0,N941UA,BOS,ORD,0,0,0,0,0
2,1,30,1603.0,1610,-7.0,1741.0,1805,-24.0,N342UA,BOS,ORD,0,0,0,0,0
3,1,31,1556.0,1605,-9.0,1726.0,1759,-33.0,N326UA,BOS,ORD,0,0,0,0,0
4,1,2,1934.0,1900,34.0,2235.0,2232,3.0,N902UA,ORD,BOS,0,0,0,0,0


### Remove cancelled and diverted flights

In [5]:
df = df[(df['Cancelled'] == 0) & (df['Diverted'] == 0)]
df.drop(['Cancelled', 'Diverted'], axis=1, inplace=True)
df.tail()

,Month,DayofMonth,DepTime,CRSDepTime,DepDelay,ArrTime,CRSArrTime,ArrDelay,TailNum,Origin,Dest,CarrierDelay,WeatherDelay,LateAircraftDelay
7140591,12,22,1652.0,1655,-3.0,1815.0,1837,-22.0,N109DL,ATL,ONT,0,0,0
7140592,12,22,1825.0,1825,0.0,1858.0,1848,10.0,N932DL,ATL,MEM,0,0,0
7140593,12,22,1507.0,1511,-4.0,1641.0,1649,-8.0,N306DL,ATL,SAT,0,0,0
7140594,12,22,924.0,925,-1.0,1056.0,1111,-15.0,N981DL,ATL,MSP,0,0,0
7140595,12,22,1345.0,1344,1.0,1621.0,1614,7.0,N3749D,ORD,SLC,0,0,0


In [14]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6992838 entries, 0 to 7140595
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Month              6992838 non-null  int64  
 1   DayofMonth         6992838 non-null  int64  
 2   DepTime            6992838 non-null  float64
 3   CRSDepTime         6992838 non-null  int64  
 4   DepDelay           6992838 non-null  float64
 5   ArrTime            6992838 non-null  float64
 6   CRSArrTime         6992838 non-null  int64  
 7   ArrDelay           6992838 non-null  float64
 8   TailNum            6992838 non-null  object 
 9   Origin             6992838 non-null  object 
 10  Dest               6992838 non-null  object 
 11  CarrierDelay       6992838 non-null  int64  
 12  WeatherDelay       6992838 non-null  int64  
 13  LateAircraftDelay  6992838 non-null  int64  
dtypes: float64(4), int64(7), object(3)
memory usage: 800.3+ MB


### For detecting cascade delays we shall use `TailNum` column

In [16]:
df['TailNum'].value_counts()

N308SW    5344
N480HA    3701
N478HA    3674
N487HA    3642
N486HA    3613
          ... 
N187UA       5
N163AT       2
N161AT       2
N8913E       2
N663US       1
Name: TailNum, Length: 5047, dtype: int64

### The flight will be considered as delayed if `DepDelay > 15`

In [26]:
cols = ['Month', 'DayofMonth', 'DepTime', 'ArrTime', 'DepDelay', 'Origin', 'Dest', 'LateAircraftDelay']
df[(df['TailNum'] == 'N486HA') & (df['DepDelay'] > 15)].sort_values(by=['Month', 'DayofMonth', 'DepTime'])[cols]

,Month,DayofMonth,DepTime,ArrTime,DepDelay,Origin,Dest,LateAircraftDelay
366861,1,20,1932.0,2014.0,34.0,KOA,HNL,0
365190,1,21,627.0,708.0,57.0,HNL,LIH,0
365216,1,21,726.0,802.0,51.0,LIH,HNL,51
365473,1,21,825.0,859.0,20.0,HNL,OGG,17
366620,1,21,926.0,1017.0,16.0,OGG,LIH,16
931193,2,12,1821.0,1859.0,16.0,HNL,LIH,16
1520149,3,4,1605.0,1652.0,20.0,HNL,ITO,15
1520118,3,4,1723.0,1805.0,18.0,ITO,HNL,0
1520259,3,4,1848.0,1921.0,23.0,HNL,KOA,0
1520088,3,5,1808.0,1853.0,23.0,HNL,KOA,19


### Just iterate over each row and store all cascade failures with departure delays (surely not the most effective way to do it)

In [33]:
def cascades(df, flight_num, delay_threshold=15):
    result = []
    current_path = []
    current_delays = []
    current_month = None
    current_day = None
    for index, row in df[(df['TailNum'] == flight_num) & (df['DepDelay'] > delay_threshold)].iterrows():
        if not current_path:
            current_path.append(row['Origin'])
            current_path.append(row['Dest'])
            current_delays.append(row['DepDelay'])
            current_month = row['Month']
            current_day = row['DayofMonth']
        else:
            if row['Month'] == current_month and row['DayofMonth'] == current_day and row['Origin'] == current_path[-1]:
                current_path.append(row['Dest'])
                current_delays.append(row['DepDelay'])
            else:
                if len(current_delays) > 1:
                    result.append((tuple(current_path), tuple(current_delays)))
                current_path = []
                current_delays = []
    if len(current_delays) > 1:
        result.append((tuple(current_path), tuple(current_delays)))
    return result

In [42]:
# NB! This loop can take up to 1 hour
all_cascades = []
for tail_num in df['TailNum'].unique():
    casc = cascades(df, tail_num)
    print(tail_num, len(casc))
    if len(casc) > 0:
        all_cascades.extend(casc)

N935UA 11
N941UA 10
N342UA 11
N326UA 8
N902UA 7
N904UA 7
N942UA 7
N920UA 2
N340UA 10
N929UA 8
N934UA 6
N336UA 11
N923UA 10
N917UA 2
N906UA 4
N932UA 10
N910UA 9
N927UA 8
N919UA 8
N203UA 10
N392UA 1
N916UA 1
N312UA 6
N912UA 9
N348UA 10
N363UA 1
N333UA 5
N349UA 7
N343UA 7
N347UA 6
N398UA 11
N921UA 8
N330UA 10
N314UA 15
N338UA 12
N346UA 8
N318UA 6
N309UA 7
N310UA 8
N315UA 7
N313UA 9
N381UA 9
N311UA 8
N325UA 9
N908UA 0
N903UA 11
N323UA 1
N855UA 4
N839UA 1
N405UA 6
N433UA 12
N822UA 5
N817UA 7
N835UA 10
N801UA 3
N806UA 11
N402UA 7
N811UA 4
N840UA 4
N842UA 10
N853UA 3
N831UA 8
N829UA 4
N803UA 9
N411UA 6
N810UA 6
N833UA 8
N824UA 10
N421UA 4
N844UA 7
N838UA 5
N814UA 5
N841UA 3
N848UA 4
N846UA 4
N455UA 8
N332UA 11
N918UA 12
N425UA 4
N304UA 7
N327UA 10
N462UA 4
N301UA 9
N924UA 10
N914UA 14
N362UA 1
N365UA 7
N419UA 3
N331UA 7
N386UA 9
N843UA 9
N832UA 6
N837UA 9
N819UA 5
N416UA 4
N461UA 8
N582UA 5
N456UA 9
N429UA 4
N584UA 6
N422UA 4
N427UA 1
N436UA 4
N463UA 4
N418UA 10
N516UA 5
N408UA 4
N417UA 6
N42

N621SW 31
N450 24
N777 25
N431 25
N778 25
N386SW 36
N772 28
N460 22
N731 28
N776 21
N317SW 27
N678 47
N324 42
N484 29
N729SW 23
N410 35
N375 44
N664WN 48
N634SW 30
N725 23
N756 37
N700GS 29
N673 44
N723 33
N513 26
N480 34
N795 23
N738 21
N479 36
N632SW 39
N623SW 24
N773 25
N754 24
N504 32
N705SW 25
N471 22
N392SW 40
N494 21
N749SW 27
N694SW 40
N403 27
N656SW 39
N697SW 38
N432 33
N611SW 26
N635SW 31
N465 35
N423 25
N693SW 40
N312 32
N684 40
N633SW 35
N322 41
N653SW 39
N300SW 41
N659SW 43
N373 40
N463 28
N328SW 41
N674AA 47
N404 25
N715SW 24
N482 32
N452 23
N474 22
N308SW 87
N618WN 33
N636WN 42
N683 39
N320 52
N368 52
N360 45
N399WN 44
N429 22
N521 36
N310 33
N413 30
N782 24
N649SW 37
N363 41
N439 23
N485 35
N466 30
N343 39
N524 37
N716SW 23
N679 45
N663SW 49
N402 24
N335 45
N750 31
N461 30
N448 24
N457 20
N706SW 20
N443 29
N456 25
N710SW 33
N405 25
N421 25
N755 24
N492 15
N600WN 40
N627SW 42
N369 29
N794 32
N784 34
N612SW 30
N771 30
N389SW 29
N688SW 34
N732 36
N356 54
N739 31
N344 41
N7

N292UX 58
N214SW 51
N232SW 40
N218SW 46
N580SW 58
N226SW 43
N229SW 53
N270YV 58
N289YV 50
N581SW 64
N250YV 48
N295UX 52
N288SW 64
N560SW 44
N221SW 57
N291SW 46
N251YV 64
N268UE 34
N561SW 57
N393SW 62
N234SW 47
N237SW 38
N293SW 63
N937SW 44
N703SK 49
N227SW 53
N271YV 57
N236SW 55
N186SW 50
N213SW 63
N217SW 49
N224SW 58
N912SW 53
N565SW 54
N297SW 51
N585SW 52
N290SW 59
N295SW 67
N916SW 51
N709SK 40
N920SW 50
N913SW 42
N954SW 48
N935SW 44
N710SK 47
N963SW 41
N953SW 54
N713SK 41
N906SW 42
N975SW 40
N927SW 63
N925SW 45
N930SW 49
N941SW 39
N967SW 53
N705SK 40
N706SK 43
N908SW 58
N903SW 65
N957SW 44
N929SW 56
N970SW 31
N709BR 34
N918SW 41
N962SW 48
N926SW 57
N905SW 58
N936SW 49
N938SW 42
N716SK 41
N943SW 46
N715SK 47
N917SW 49
N939SW 58
N924SW 52
N980SW 47
N946SW 43
N976SW 37
N443SW 41
N702SK 45
N944SW 50
N982SW 49
N909SW 51
N973SW 41
N945SW 37
N699BR 27
N978SW 48
N948SW 41
N907SW 47
N983SW 46
N910SW 53
N969SW 41
N928SW 52
N986SW 39
N956SW 57
N947SW 44
N958SW 41
N961SW 48
N934SW 71
N979SW 43


N867DA 4
N193DN 0
N1501P 0
N156DL 0
N153DL 1
N175DN 4
N1608 3
N1613B 0
N185DN 1
N152DL 1
N176DN 0
N199DN 0
N196DN 0
N1605 2
N661DN 10
N630DL 14
N690DL 17
N652DL 19
N830MH 10
N3730B 20
N608DA 17
N116DL 16
N104DA 9
N834MH 15
N757AT 13
N122DL 16
N631DL 18
N674DL 17
N139DL 17
N642DL 18
N611DL 16
N381DA 12
N662DN 19
N395DN 24
N672DL 14
N624DL 14
N372DA 15
N637DL 14
N102DA 21
N636DL 21
N143DA 20
N682DA 10
N3751B 11
N134DL 17
N3766 7
N108DL 18
N3749D 11
N603DL 20
N670DN 17
N3768 11
N605DL 21
N311DL 20
N388DA 9
N838MH 17
N383DN 11
N3761R 15
N837MH 19
N635DL 11
N191DN 2
N678DL 17
N639DL 33
N173DN 1
N186DN 0
N169DZ 0
N1610D 1
N1611B 1
N195DN 0
N1603 2
N640DL 15
N632DL 17
N120DL 21
N1201P 2
N623DL 19
N173DZ 20
N3763D 15
N331DL 21
N862DA 6
N1604R 0
N1609 1
N177DN 0
N154DL 0
N1612T 0
N188DN 1
N189DN 0
N172DN 1
N239WA 35
N302DL 27
N909DA 12
N917DL 35
N171DZ 19
N135DL 22
N607DL 20
N3732J 15
N190DN 3
N1602 1
N184DN 1
N128DL 10
N831MH 16
N1200K 1
N655DL 30
N192DN 0
N171DN 2
N180DN 1
N312WA 10
N394DL 3


N345AA 1
N389AA 4
N399AA 1
N342AA 1
N398AA 2
N397AA 3
N368AA 2
N3BWAA 6
N3CYAA 5
N3DAAA 4
N3AHAA 4
N3APAA 3
N3ADAA 6
N3BVAA 4
N5FBAA 7
N5EDAA 7
N5ECAA 5
N5FDAA 5
N5ESAA 6
N5DKAA 10
N5DHAA 6
N5EYAA 10
N5EVAA 7
N5FAAA 2
N5DJAA 9
N5DGAA 8
N5DFAA 6
N5EUAA 4
N5EXAA 7
N5FCAA 3
N392AA 3
N610AA 3
N3BJAA 7
N3AAAA 8
N3BEAA 7
N3DDAA 5
N3BKAA 8
N3BUAA 5
N3BSAA 4
N3BAAA 6
N3AMAA 5
N3CCAA 6
N3ABAA 4
N3BNAA 4
N3ATAA 6
N3AFAA 9
N3BLAA 6
N376AA 2
N355AA 3
N393AA 3
N374AA 4
N349AA 6
N353AA 4
N380AA 2
N5ETAA 6
N5ERAA 4
N5DEAA 4
N5DLAA 4
N5DUAA 5
N642AA 1
N5FRAA 3
N643AA 11
N5BUAA 8
N5EMAA 4
N5CFAA 5
N5EGAA 4
N640AA 3
N626AA 4
N611AA 4
N638AA 7
N5EAAA 8
N629AA 1
N615AA 4
N5DCAA 1
N5EWAA 4
N7AMAA 0
N7AEAA 1
N7BCAA 2
N7AHAA 0
N7ANAA 2
N7AFAA 1
N7BWAA 1
N7ARAA 1
N7APAA 0
N7ACAA 0
N7BXAA 1
N7AUAA 0
N7ASAA 0
N409AA 8
N7AKAA 2
N7BYAA 1
N7AJAA 0
N7ADAA 2
N7ATAA 0
N7BVAA 0
N3CHAA 5
N3CTAA 6
N3ARAA 6
N3AYAA 6
N3CKAA 7
N3CPAA 6
N3CDAA 4
N3CJAA 7
N3CNAA 6
N3CXAA 4
N3AGAA 6
N3CLAA 3
N3BXAA 3
N3DFAA 5
N3ALAA 4
N3BPAA 

N12327 2
N58606 3
N37281 1
N17614 4
N12225 3
N19623 1
N15659 1
N56859 3
N33284 3
N73251 3
N46625 5
N38403 3
N12116 1
N14325 4
N14324 2
N75410 3
N16646 5
N32626 4
N12349 0
N14655 4
N14342 4
N60312 2
N76288 2
N11206 3
N23661 3
N13113 1
N12319 1
N26210 4
N13248 1
N33203 5
N14214 0
N70330 4
N33103 3
N11651 3
N27722 1
N37409 2
N37253 0
N14667 2
N18622 6
N14662 6
N71411 4
N14337 3
N25705 3
N18223 1
N73275 4
N35407 1
N31412 1
N69333 1
N17122 4
N17133 3
N27239 3
N75858 4
N33266 4
N33132 5
N33292 1
N12216 3
N61304 3
N16617 6
N76265 3
N75853 7
N12322 5
N11641 4
N18243 3
N14605 6
N37290 3
N16301 7
N17245 1
N13720 1
N14231 6
N14639 5
N19357 1
N37018 0
N14230 4
N17356 3
N57111 1
N16632 6
N16618 7
N17627 5
N16648 6
N76054 2
N76354 2
N17326 3
N17620 5
N47332 0
N17233 3
N27213 0
N14735 1
N73256 3
N15710 0
N54241 3
N75854 2
N37255 4
N14347 5
N37267 4
N34315 1
N13716 3
N16701 2
N27205 2
N17328 0
N26208 6
N36280 1
N18220 2
N14609 4
N14115 3
N36272 2
N17719 2
N14308 7
N17644 6
N24633 6
N37287 2
N77258 0
N

In [84]:
import pickle

with open("cascades.pickle", 'wb') as f:
    pickle.dump(all_cascades, f)

### Store information about mean delays and paths in a dict

In [66]:
cascades_dict = dict()
for path, delays in all_cascades:
    for i in range(len(delays)):
        if (path[i], path[i+1]) not in cascades_dict:
            cascades_dict[(path[i], path[i+1])] = (delays[i], len(delays) - i, 1)
        else:
            prev_mean, prev_len, n = cascades_dict[(path[i], path[i+1])]
            n += 1
            new_mean = prev_mean + (delays[i] - prev_mean) / n
            new_len = prev_len + (len(delays) - i - prev_len) / n
            cascades_dict[(path[i], path[i+1])] = (new_mean, new_len, n)

In [68]:
len(cascades_dict)

4059

### Store airports coordinates in a dict

In [56]:
iata2coords = dict()
for index, row in airport_df.iterrows():
    iata2coords[row['iata']] = (row['lat'], row['long'])

print(len(iata2coords))

3376


### Create an empty graph

In [75]:
G = nx.DiGraph()
len(G)

0

### Add airports as nodes to the graph

In [76]:
airports = np.union1d(df['Origin'].unique(), df['Dest'].unique())
G.add_nodes_from(airports)
print(len(G), len(G.edges()))

285 0


### Add edges to the graph

In [77]:
for route, stats in cascades_dict.items():
    G.add_edge(route[0], route[1], mean_delay=stats[0], mean_length=round(stats[1], 3))

In [78]:
print(len(G), len(G.edges()))

285 4059


In [79]:
c = 0
for u, v, d in G.edges(data=True):
    print(u, v, d)
    c += 1
    if c > 10:
        break

ABE CLT {'mean_delay': 25.0, 'mean_length': 2}
ABE PHL {'mean_delay': 131.33333333333334, 'mean_length': 2.0}
ABE DTW {'mean_delay': 121.66666666666667, 'mean_length': 2.333}
ABE ATL {'mean_delay': 70.61428571428573, 'mean_length': 1.871}
ABE CVG {'mean_delay': 69.90625, 'mean_length': 1.438}
ABE ORD {'mean_delay': 82.6, 'mean_length': 2.1}
ABE CLE {'mean_delay': 29.666666666666668, 'mean_length': 1.333}
ABI IAH {'mean_delay': 56.230769230769226, 'mean_length': 1.846}
ABI DFW {'mean_delay': 50.20588235294118, 'mean_length': 1.5}
ABQ DEN {'mean_delay': 54.848214285714285, 'mean_length': 2.08}
ABQ ORD {'mean_delay': 56.73913043478261, 'mean_length': 1.696}


### Plotly showtime

In [98]:
for edge in G.edges():
    print(edge)
    break

('ABE', 'CLT')


In [103]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = iata2coords[edge[0]]
    x1, y1 = iata2coords[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.1, color='#888'),
    hoverinfo='none',
    mode='lines')

In [115]:
node_x = []
node_y = []
sizes = []
colors = []
texts = []
for count, delay, length in zip(G.in_degree(),G.in_degree(weight='mean_delay'), G.in_degree(weight='mean_length')):
    assert count[0] == length[0]
    if count[1] > 0:
        x, y = iata2coords[count[0]]
        texts.append(count[0])
        node_x.append(x)
        node_y.append(y)
        colors.append(delay[1] / count[1])
        sizes.append(5 * length[1] / count[1])

In [118]:
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=texts,
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Earth',
        reversescale=True,
        color=colors,
        size=sizes,
        colorbar=dict(
            thickness=15,
            title='Average delay',
            xanchor='left',
            titleside='right'
        ),
        line_width=1))

In [120]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Cascading failures',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()